In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
av_data = pd.read_csv("Data Part2/mp1_av_disengagements.csv")
print(av_data.shape)
av_data.head()

(1024, 7)


,Month,Car,Location,Weather,TypeOfTrigger,ReactionTime,Cause
0,14-Sep,WDD2221851A001072,urban-street,cloudy,automatic,0.58,Incorrect Traffic Light Detection
1,14-Sep,WDD2221851A001072,urban-street,cloudy,automatic,0.66,Adverse road surface conditions
2,14-Sep,WDD2221851A001072,urban-street,cloudy,automatic,1.93,Position Estimation Failure
3,14-Sep,WDD2221851A001072,urban-street,cloudy,automatic,0.65,Emergency Vehicle
4,14-Sep,WDD2221851A001072,urban-street,cloudy,automatic,0.64,Adverse road surface conditions


## Task 3

Given below are some assumptions that you will need to do the analysis for this task.
1. There can be at most one disengagement per mile.
2. The total amount of miles driven by all the AVs in the dataset is 505229 miles.
3. A day can be either clear or cloudy, but not both. The probability of a day being
clear in California is 72% [7].
4. The AV is equally likely to drive on a cloudy day as on a clear day
<br>
<br>
__________________________________________________
<br>

**1. Based on the above assumptions, answer the following questions on basic probability. (6 points)**

>**a. The assumption of at most one disengagement per mile allows us to treat
the occurrence of a disengagement in a mile as a random variable with a
__ __BERNOULLI__ __ distribution**

>**b. Based on the above assumptions, calculate the probability of
disengagement per mile on a cloudy day. (1 point)**

In [10]:
## 1024 disengagements / 505229 miles
p_disengagement = 1024/505229
p_cloudy_given_disengagement = av_data[av_data['Weather']=='cloudy'].shape[0] / 1024

## P(Disengagement | Cloudy) = P(Cloudy | Disengagement) * P(Disengagement) / P(Cloudy)
p_disengagement_given_cloudy = (p_cloudy_given_disengagement*p_disengagement)/(1-0.72)
print("Probability of Disengagement Per Mile: ",p_disengagement)
print("Probability of Disengagement Per Mile on a Cloudy Day: ",p_disengagement_given_cloudy)

Probability of Disengagement Per Mile:  0.002026803687040926
Probability of Disengagement Per Mile on a Cloudy Day:  0.00590255677552725


>**c. Based on the above assumptions, calculate the probability of
disengagement per mile on a clear day. (1 point)**

In [11]:
p_clear_given_disengagement = av_data[av_data['Weather']=='clear'].shape[0] / 1024

p_disengagement_given_clear = (p_clear_given_disengagement*p_disengagement)/(0.72)
print("Probability of Disengagement Per Mile on a Clear Day: ",p_disengagement_given_clear)

Probability of Disengagement Per Mile on a Clear Day:  0.0005195663748517999


>**d. Similarly, calculate the probability of an automatic disengagement per mile
on a cloudy day, and the probability of an automatic disengagement per mile on a
clear day. (1 point)**

In [13]:
automatic_disengagements = av_data[av_data['TypeOfTrigger']=='automatic']

p_automatic_disengagement = automatic_disengagements.shape[0] / 505229
p_cloudy_given_automatic_disengagement = automatic_disengagements[automatic_disengagements['Weather']=='cloudy'].shape[0]/  \
                                            automatic_disengagements.shape[0]
p_clear_given_automatic_disengagement = automatic_disengagements[automatic_disengagements['Weather']=='clear'].shape[0]/  \
                                            automatic_disengagements.shape[0]
                                            
p_automatic_disengagement_given_cloudy = (p_cloudy_given_automatic_disengagement*p_automatic_disengagement)/(1-0.72)
p_automatic_disengagement_given_clear = (p_clear_given_automatic_disengagement*p_automatic_disengagement)/(0.72)

print("Probability of Automatic Disengagement Per Mile: ",p_automatic_disengagement)
print("Probability of Automatic Disengagement Per Mile on Cloudy Day: ",p_automatic_disengagement_given_cloudy)
print("Probability of Automatic Disengagement Per Mile on Cloudy Day: ",p_automatic_disengagement_given_clear)

Probability of Automatic Disengagement Per Mile:  0.0009757951344835708
Probability of Automatic Disengagement Per Mile on Cloudy Day:  0.0028063653172267283
Probability of Automatic Disengagement Per Mile on Cloudy Day:  0.00026390673008345393


>**e. How likely is it that there are 100 or more disengagements in 10,000 miles
under cloudy conditions? (hint: use Central Limit Theorem) (2 points)**

In [14]:
import scipy

Given Random variable $X_i$ ~ Bernoulli($p = 0.0059$), we know that as $n \rightarrow \infty$:

$P(\frac{\sum{X_i} - n\mu}{\sigma\sqrt{n}} \leq a) = \frac{1}{\sqrt{2\pi}} \int_{-\infty}^{a} e^{\frac{-x^2}{2}} dx$ 

We want: $P(X_1 + ... + X_{10000} \geq 100) = 1 - P(X_1 + ... + X_{10000} < 99)$

To achieve the form above, have that: $\sum(X_i) <= (\sigma\sqrt{n})a + n\mu = 99$ 

With $n=10000, \ \mu = p = 0.0059, \ \sigma = \sqrt{p(1-p)} = 0.0766$

In [38]:
import scipy.integrate

In [61]:
n = 10000
p = 0.0059
sigma = 0.0766

A = (99 - (n*p))/(sigma*np.sqrt(n))
print("a =",A)

prob = 1 - ( (1/np.sqrt(2*np.pi)) * scipy.integrate.quad(func=lambda x: np.exp((x**2)/-2),
                                                  a=-np.inf,b=A)[0] )
print("Probability of 100 or more disengagements in 10000 miles =",round(prob,3))

a = 5.221932114882507
Probability of 100 or more disengagements in 10000 miles = 0.0


**2. Assuming that the disengagement per mile is a random variable with the
distribution you answered in Task 3.1.a, and the weather condition is cloudy. (6
points)**

>**a. What is the distribution of “the number of miles until the next
disengagement”? Explain your reasoning. Calculate and state the values of the
parameters of the distribution. (1 points)**

>**b. What is the distribution of “the number of disengagements in 10,000 miles”?
(hint: this is equivalent to drawing n=10,000 independent trials from the
distribution of disengagement per mile you calculated from Task 3.1.a). Calculate
and state the values of the parameters of the distribution. (1 points)**

>**c. Notice that the number of disengagements “𝒏" in Task 3.2.b is large while
the probability of disengagement per mile "𝒑" is very small, what distribution does
your answer in Task 3.2.b approximate? Calculate and state the values of the
parameters of the distribution. (1 points)**

>**d. Plot the probability mass function (PMF) of the distribution in Task 3.2.b and
Task 3.2.c for: 1 - The x-axis ranging between 0 and 10000. 2 - The x-axis ranging between 0 and 100.
Describe the plots and discuss your findings. (2 points)**

>**e. Solve Task 3.1.e by using the cumulative distribution function (CDF) of the
distribution you computed in Task 3.2.c and compare the results. Discuss your
findings. (1 points)**